# Определение источников транзакций

<b>Описание:</b>

Внутри BigQuery имеется публичный датасет ga_sessions с описанием визитов пользователей на сайт интернет-магазина. Необходимо сформировать выгрузку за период 2 недели и для каждой совершенной транзакции составить цепочку источников визитов пользователя. Также необходимо определить топ-10 самых популярных цепочек. 

<b>Цель:</b>
1. Для каждой транзакции определить цепочку касаний.
2. Выявить топ-10 самых популярных цепочек касания, приведших к транзакции.

## Импорт библиотек и предобработка данных

Импортируем необходимые библиотеки и устанавливаем настройки

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import warnings

warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Загружаем таблицу

Таблица сформирована SQL-запросом:

SELECT DISTINCT fullVisitorId,

       visitNumber,        
       visitId, 
       visitStartTime, 
       date, 
       trafficSource.source,
       trafficSource.medium,
       hits.transaction.transactionId
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`,

UNNEST(hits) AS hits

WHERE _table_suffix BETWEEN '20170719' AND '20170801'

In [2]:
df = pd.read_csv('bq-results-20230205-230025-1675638046829.csv', 
                dtype={'fullVisitorId':str,
                       'visitNumber':int,
                       'visitId':str,
                       'visitStartTime':str,
                       'date':str,
                       'source':str,
                       'medium':str,
                       'transactionId':str})
df.head()

,fullVisitorId,visitNumber,visitId,visitStartTime,date,source,medium,transactionId
0,2265977830656365501,1,1500821038,1500821038,20170723,google,organic,NaN
1,6474095028710839893,2,1500875167,1500875167,20170723,google,organic,NaN
2,6443653150131206845,3,1500822280,1500822280,20170723,google,organic,NaN
3,9860085665958246812,2,1500831228,1500831228,20170723,google,organic,NaN
4,7700344873342427798,4,1500816531,1500816531,20170723,google,organic,NaN


In [3]:
print(f'Дубликатов в таблице: {df.duplicated().sum()}', end='\n\n')
df.info()

Дубликатов в таблице: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33146 entries, 0 to 33145
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   fullVisitorId   33146 non-null  object
 1   visitNumber     33146 non-null  int32 
 2   visitId         33146 non-null  object
 3   visitStartTime  33146 non-null  object
 4   date            33146 non-null  object
 5   source          33146 non-null  object
 6   medium          33146 non-null  object
 7   transactionId   521 non-null    object
dtypes: int32(1), object(7)
memory usage: 1.9+ MB


Дубликатов в таблице нет (так как мы выгрузили DISTINCT значения). Приведем данные к нужным форматам.

In [4]:
df['visitStartTime'] = pd.to_datetime(df['visitStartTime'], unit='s')
df['date'] = pd.to_datetime((df['date']), format='%Y%m%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33146 entries, 0 to 33145
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   fullVisitorId   33146 non-null  object        
 1   visitNumber     33146 non-null  int32         
 2   visitId         33146 non-null  object        
 3   visitStartTime  33146 non-null  datetime64[ns]
 4   date            33146 non-null  datetime64[ns]
 5   source          33146 non-null  object        
 6   medium          33146 non-null  object        
 7   transactionId   521 non-null    object        
dtypes: datetime64[ns](2), int32(1), object(5)
memory usage: 1.9+ MB


In [5]:
df.head()

,fullVisitorId,visitNumber,visitId,visitStartTime,date,source,medium,transactionId
0,2265977830656365501,1,1500821038,2017-07-23 14:43:58,2017-07-23,google,organic,NaN
1,6474095028710839893,2,1500875167,2017-07-24 05:46:07,2017-07-23,google,organic,NaN
2,6443653150131206845,3,1500822280,2017-07-23 15:04:40,2017-07-23,google,organic,NaN
3,9860085665958246812,2,1500831228,2017-07-23 17:33:48,2017-07-23,google,organic,NaN
4,7700344873342427798,4,1500816531,2017-07-23 13:28:51,2017-07-23,google,organic,NaN


Проверяем, что даты в таблице те, что мы хотели получить.

In [6]:
print(f'Минимальная дата visitStartTime: {df["visitStartTime"].min()}')
print(f'Минимальная дата visitStartTime: {df["visitStartTime"].max()}')
print(f'Минимальная дата date: {df["date"].min()}')
print(f'Минимальная дата date: {df["date"].max()}')

Минимальная дата visitStartTime: 2017-07-19 07:00:27
Минимальная дата visitStartTime: 2017-08-02 06:59:53
Минимальная дата date: 2017-07-19 00:00:00
Минимальная дата date: 2017-08-01 00:00:00


Данные предобработаны, приступаем к исследовательскому анализу.

## Исследовательский анализ и подготовка финального датасета 

### Траназакции

Проверяем, что нет дублирующихся транзакций.

In [7]:
print(f'Всего транзакций: {df["transactionId"].count()}')
print(f'Уникальных транзакций: {df["transactionId"].nunique()}')

Всего транзакций: 521
Уникальных транзакций: 521


Создадим столбец `is_transaction` с флагом, что в данной строке есть транзакция.

In [8]:
df['transactionId'] = df['transactionId'].astype(str)
df['is_transaction'] = df['transactionId'].apply(lambda x: 1 if re.search('.*\d+.*', x) else 0)
df['transactionId'] = df['transactionId'].replace('nan', np.nan)
df['is_transaction'].value_counts()

0    32625
1      521
Name: is_transaction, dtype: int64

### Визиты

Смотрим, какие по счету визиты есть в таблице.

In [9]:
df['visitNumber'].value_counts().head()

1    24195
2     3911
3     1588
4      888
5      553
Name: visitNumber, dtype: int64

Необходимо в финальном датасете оставить только пользователей, совершивших транзакцию. А также только пользователей, для которых можно составить цепочку с первого визита. 

То есть пользователи, которые ранее 19.07.2017 совершали визиты, не попали к нам в таблицу, и корректную цепочку для них мы составить не можем.

In [10]:
good_visitors = df.query('is_transaction == 1')['fullVisitorId'].unique()
print(f'Всего пользователей, совершивших транзакцию: {len(good_visitors)}')

Всего пользователей, совершивших транзакцию: 492


In [11]:
print(f'Длина таблицы до: {len(df)}')
df_good = df.query('fullVisitorId in @good_visitors')
print(f'Длина таблицы после: {len(df_good)}')

Длина таблицы до: 33146
Длина таблицы после: 1730


Определяем минимальный порядковый номер визита.

In [12]:
min_visit = df_good.groupby('fullVisitorId').agg({'visitNumber':'min'}).reset_index()
min_visit.head()

,fullVisitorId,visitNumber
0,0010295111715775250,2
1,0024932550342595467,2
2,0047078955120420928,1
3,0080479763428955064,3
4,0082806901961150595,1


In [13]:
good_visitors_2 = min_visit.query('visitNumber == 1')['fullVisitorId'].unique()
print('Всего пользователей, совершивших транзакцию и о которых имеются данные с первого захода: {}'
     .format(len(good_visitors_2)))

Всего пользователей, совершивших транзакцию и о которых имеются данные с первого захода: 286


В финальном датафрейме `df_fin` оставляем только пользователей, для которых есть данные с первого захода и которые совершили транзакцию.

In [14]:
df_fin = df_good.query('fullVisitorId in @good_visitors_2')
print(f'Длина итоговой таблицы для анализа: {len(df_fin)}', end='\n\n')
print(f'Количество транзакций в таблице: {df_fin["transactionId"].count()}')
print(f'Уникальных пользователей в таблице: {df_fin["fullVisitorId"].nunique()}')
df_fin.head()

Длина итоговой таблицы для анализа: 966

Количество транзакций в таблице: 297
Уникальных пользователей в таблице: 286


,fullVisitorId,visitNumber,visitId,visitStartTime,date,source,medium,transactionId,is_transaction
9,9973665079624172058,1,1500870192,2017-07-24 04:23:12,2017-07-23,(direct),(none),NaN,0
574,3563510904717248245,3,1500853402,2017-07-23 23:43:22,2017-07-23,google,organic,NaN,0
1381,190492934318255272,6,1500860330,2017-07-24 01:38:50,2017-07-23,google,organic,NaN,0
1382,5322397769480961983,4,1500857903,2017-07-24 00:58:23,2017-07-23,google,organic,NaN,0
1546,599110551792149234,4,1500851005,2017-07-23 23:03:25,2017-07-23,(direct),(none),NaN,0


### Источники

In [15]:
df_fin['source'].value_counts()

(direct)            542
google              405
sites.google.com     10
mail.google.com       5
youtube.com           4
Name: source, dtype: int64

Удалим окончание `.com` из названия источника, а также скобки при прямом заходе.

In [16]:
df_fin['source'] = df_fin['source'].apply(lambda x:  re.sub('(\.com$)', '', x))
df_fin['source'] = df_fin['source'].replace('(direct)', 'direct')
df_fin['source'].value_counts()

direct          542
google          405
sites.google     10
mail.google       5
youtube           4
Name: source, dtype: int64

In [17]:
df_fin['medium'].value_counts()

(none)      542
organic     329
cpc          60
referral     19
cpm          16
Name: medium, dtype: int64

Тоже удалим скобки для прямых заходов.

In [18]:
df_fin['medium'] = df_fin['medium'].replace('(none)', 'none')
df_fin['medium'].value_counts()

none        542
organic     329
cpc          60
referral     19
cpm          16
Name: medium, dtype: int64

Добавим колонку с объединенными source и medium составляющими источника.

In [19]:
df_fin['source_medium'] = df_fin['source'] + '_' + df_fin['medium']
df_fin['source_medium'].value_counts()

direct_none              542
google_organic           329
google_cpc                60
google_cpm                16
sites.google_referral     10
mail.google_referral       5
youtube_referral           4
Name: source_medium, dtype: int64

Видим, что чаще всего источником визита был прямой заход, а не по размеченным источникам.

In [20]:
df_fin = df_fin.sort_values(['fullVisitorId', 'visitStartTime'])
df_fin.head()

,fullVisitorId,visitNumber,visitId,visitStartTime,date,source,medium,transactionId,is_transaction,source_medium
33023,0047078955120420928,1,1501541991,2017-07-31 22:59:51,2017-07-31,direct,none,NaN,0,direct_none
33024,0047078955120420928,1,1501541991,2017-07-31 22:59:51,2017-07-31,direct,none,ORD201707312556,1,direct_none
25246,0082806901961150595,1,1501008172,2017-07-25 18:42:52,2017-07-25,direct,none,NaN,0,direct_none
25247,0082806901961150595,1,1501008172,2017-07-25 18:42:52,2017-07-25,direct,none,ORD201707252631,1,direct_none
11995,024507252193437459,1,1501250419,2017-07-28 14:00:19,2017-07-28,direct,none,NaN,0,direct_none


##### Вывод:
- Подготовлен финальный датасет для анализа источников транзакций
- В нем 966 строк и 297 транзакций 
- 286 уникальных пользователей, то есть имеются пользователи, совершившие не одну транзакцию 
- Самым популярным источником визита является прямой заход.

## Определение цепочек для транзакций

Составим таблицу, где для каждого пользователя, визита и транзакции будет указан источник без дубликатов.

In [21]:
# уникальные номера визитов для пользователя
visit_source = df_fin[['fullVisitorId', 'visitNumber', 'source_medium']].drop_duplicates()

# уникальные сочетания номера пользователя, номера визита, номера транзакции
visit_transactions = df_fin[['fullVisitorId', 'visitNumber',
        'transactionId', 'is_transaction']].drop_duplicates()

# оставляем для каждого номера визита с транзакцией только запись, где была транзакция 
# (т.к. источник для каждого визита единственный)
visit_transactions = visit_transactions.query('is_transaction == 1')

# и соединяем таблицы
df_transactions = visit_source.merge(visit_transactions, 
                                     on=['fullVisitorId', 'visitNumber'], 
                                     how='left')

df_transactions['is_transaction'] = df_transactions['is_transaction'].fillna(0).astype(int)
df_transactions.head()

,fullVisitorId,visitNumber,source_medium,transactionId,is_transaction
0,0047078955120420928,1,direct_none,ORD201707312556,1
1,0082806901961150595,1,direct_none,ORD201707252631,1
2,024507252193437459,1,direct_none,NaN,0
3,024507252193437459,2,direct_none,NaN,0
4,024507252193437459,3,mail.google_referral,ORD201708012322,1


Составим списки источников, транзакций для каждого пользователя.

In [22]:
df_grouped_transactions = (df_transactions
                           .groupby('fullVisitorId')
                           .agg({'source_medium': lambda x: list(x), 
                                 'transactionId': lambda x: list(x),
                                 'is_transaction': lambda x: list(x)}))

df_grouped_transactions.head()

,source_medium,transactionId,is_transaction
fullVisitorId,,,
0047078955120420928,[direct_none],[ORD201707312556],[1]
0082806901961150595,[direct_none],[ORD201707252631],[1]
024507252193437459,"[direct_none, direct_none, mail.google_referral]","[nan, nan, ORD201708012322]","[0, 0, 1]"
0249659095832018179,[direct_none],[ORD201707242363],[1]
0316080058721038988,"[google_cpm, google_organic, google_organic]","[nan, ORD201707192364, nan]","[0, 1, 0]"


Добавим поле `N_of_visit` с индексом транзакции

In [23]:
df_grouped_transactions['N_of_visit'] = (df_grouped_transactions['is_transaction']
                                         .apply(lambda x: np.where(np.array(x) == 1)[0]))

df_grouped_transactions['N_of_visit'].value_counts()

[0]             176
[1]              55
[2]              22
[3]              13
[4]               5
[5]               3
[7]               3
[0, 1]            1
[1, 7]            1
[0, 1, 2, 3]      1
[1, 2, 4]         1
[2, 3]            1
[0, 1]            1
[16]              1
[0, 1]            1
[4, 5]            1
Name: N_of_visit, dtype: int64

Чаще всего транзация совершалась в первый визит (индекс=0).

In [24]:
visits_rating = df_fin.query('is_transaction == 1')['visitNumber'].value_counts().to_frame()
visits_rating['%_of_all'] = round(visits_rating['visitNumber'] * 100 / visits_rating['visitNumber'].sum(), 1)
visits_rating

,visitNumber,%_of_all
1,180,60.6
2,61,20.5
3,24,8.1
4,14,4.7
5,7,2.4
6,6,2.0
8,4,1.3
17,1,0.3


Добавим поля для дальнейшей обработки.

In [25]:
# Список цепочек источников для каждой транзакции (если у пользователя было 2 транзакции, то
# в списке будет 2 цепочки)
df_grouped_transactions['list_source'] = (df_grouped_transactions
                                          .apply(lambda x: 
        [x['source_medium'][:x['N_of_visit'][i] + 1] for i in range(len(x['N_of_visit']))], axis=1))

# Список транзакций
df_grouped_transactions['list_transactions'] = (df_grouped_transactions
                                                .apply(lambda x: 
        [x['transactionId'][x['N_of_visit'][i]] for i in range(len(x['N_of_visit']))], axis=1))

# Список из идущих подряд уникальных источников и без прямых заходов
df_grouped_transactions['source_short'] = (df_grouped_transactions
                                         .apply(lambda x: 
        [list(filter(lambda y: y != 'direct_none', 
                     [x['list_source'][j][i] for i in range(len(x['list_source'][j])) 
          if i == 0 or (i > 0 and x['list_source'][j][i] != x['list_source'][j][i-1])
         ]))
         for j in range(len(x['N_of_visit']))
         
        ], axis=1))
                                         
df_grouped_transactions.head()                            

,source_medium,transactionId,is_transaction,N_of_visit,list_source,list_transactions,source_short
fullVisitorId,,,,,,,
0047078955120420928,[direct_none],[ORD201707312556],[1],[0],[[direct_none]],[ORD201707312556],[[]]
0082806901961150595,[direct_none],[ORD201707252631],[1],[0],[[direct_none]],[ORD201707252631],[[]]
024507252193437459,"[direct_none, direct_none, mail.google_referral]","[nan, nan, ORD201708012322]","[0, 0, 1]",[2],"[[direct_none, direct_none, mail.google_referral]]",[ORD201708012322],[[mail.google_referral]]
0249659095832018179,[direct_none],[ORD201707242363],[1],[0],[[direct_none]],[ORD201707242363],[[]]
0316080058721038988,"[google_cpm, google_organic, google_organic]","[nan, ORD201707192364, nan]","[0, 1, 0]",[1],"[[google_cpm, google_organic]]",[ORD201707192364],"[[google_cpm, google_organic]]"


In [26]:
df_grouped_transactions.query('list_transactions.str.len() > 1').head(2)

,source_medium,transactionId,is_transaction,N_of_visit,list_source,list_transactions,source_short
fullVisitorId,,,,,,,
4042880551685271731,"[direct_none, direct_none]","[ORD201707202545, ORD201707202429]","[1, 1]","[0, 1]","[[direct_none], [direct_none, direct_none]]","[ORD201707202545, ORD201707202429]","[[], []]"
4376572204985925602,"[google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic]","[nan, ORD201707192493, nan, nan, nan, nan, nan, ORD201707192400, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]","[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 7]","[[google_organic, google_organic], [google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic, google_organic]]","[ORD201707192493, ORD201707192400]","[[google_organic], [google_organic]]"


Например, у пользователей `4042880551685271731` и `4376572204985925602` было по 2 транзакции. У первого пользователя транзакция была в первый и второй визит, у второго - во второй и восьмой.

В колонке `list_transactions` перечислены номера транзакций. 

В колонке `list_source` для каждой транзакции список источников захода. Например, для второй транзакции второго пользователя считаем цепочку источников с самого первого визита, и у него будет цепочка из 8 касаний.

Поскольку в итоге нас интересуют размеченные заходы и топ цепочек, будем считать, что несколько переходов подряд с одного источника можно сложить в один переход (неважно, сколько раз подряд пользователь перешел с одного и того же источника). Поэтому в колонке `source_short` оставлены только уникальные источники для каждой транзакции (если первая и вторая одинаковые - считаем за один источник) и удалены прямые заходы. 

In [27]:
list_transactions = df_grouped_transactions['list_transactions'].to_list()
list_sources = df_grouped_transactions['list_source'].to_list()

transactions_tuple = [(list_transactions[j][i], ' => '.join(list_sources[j][i])) 
                      for j in range(len(list_transactions)) for i in range(len(list_transactions[j]))]

transactions_dict = dict(transactions_tuple)

Итоговая таблица `transactions_chain` с цепочкой касаний для каждой транзакции.

In [28]:
transactions_chain = pd.DataFrame.from_dict(transactions_dict, orient='index').reset_index()
transactions_chain.columns = ['transactionId', 'chain']
transactions_chain.head()

,transactionId,chain
0,ORD201707312556,direct_none
1,ORD201707252631,direct_none
2,ORD201708012322,direct_none => direct_none => mail.google_referral
3,ORD201707242363,direct_none
4,ORD201707192364,google_cpm => google_organic


In [29]:
list_unique_sources = df_grouped_transactions['source_short'].to_list()

transactions_short_tuple = [(list_transactions[j][i], ' => '.join(list_unique_sources[j][i])) 
                          for j in range(len(list_transactions)) for i in range(len(list_transactions[j])) 
                          if list_unique_sources[j][i]]
transactions_short_dict = dict(transactions_short_tuple)

Таблица `transactions_chain_short` с уникальными идущими подряд источниками перехода без прямых заходов

In [30]:
transactions_chain_short = pd.DataFrame.from_dict(transactions_short_dict, orient='index').reset_index()
transactions_chain_short.columns = ['transactionId', 'chain']
transactions_chain_short.head()

,transactionId,chain
0,ORD201708012322,mail.google_referral
1,ORD201707192364,google_cpm => google_organic
2,ORD201707282299,google_organic
3,ORD201708011814,google_organic
4,ORD201707262668,google_organic


Таблица `top_chain` с самыми популярными цепочками перехода для совершения транзакций. Топ-10 не набралось, потому что исследовался небольшой период.

In [31]:
top_chain = (transactions_chain_short['chain']
             .value_counts()
             .to_frame()
             .rename(columns={'chain':'n_chains'}))
top_chain

,n_chains
google_organic,95
google_cpc,18
google_cpm,7
google_cpc => google_organic,4
mail.google_referral,2
google_cpm => google_organic,2
sites.google_referral,2
youtube_referral,1
google_organic => google_cpc => google_organic,1


## Заключение

Сформированы результирующие таблицы - таблица с цепочкой касаний для каждой транзакции `transactions_chain` и таблица с самыми популялярными цепочками касаний - `top_chain`.

Поскольку анализируемый период небольшой, получилось всего 9 уникальных цепочек (без прямых заходов).

In [32]:
transactions_chain.head()

,transactionId,chain
0,ORD201707312556,direct_none
1,ORD201707252631,direct_none
2,ORD201708012322,direct_none => direct_none => mail.google_referral
3,ORD201707242363,direct_none
4,ORD201707192364,google_cpm => google_organic


In [33]:
top_chain

,n_chains
google_organic,95
google_cpc,18
google_cpm,7
google_cpc => google_organic,4
mail.google_referral,2
google_cpm => google_organic,2
sites.google_referral,2
youtube_referral,1
google_organic => google_cpc => google_organic,1


##### Вывод
Таким образом, наиболее популярным рекламным источником является google - именно для него и его сервисов наиболее часто встречаются переходы, приведшие к транзакциям.

Также отметим, что чаще всего транзакция совершалась в первый визит - в 60,6% случаев, в 20,5% - во второй визит и в 8,1% случаев - в 3-й визит.